In [1]:
import os
from dotenv import load_dotenv

# 【硬性要求】强制重载本地环境变量，确保读取到最新的 OPENAI_API_KEY
load_dotenv(override=True)

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 实例化核心组件 (Core Components)
# 现代工程标准：显式指定模型版本和 temperature，保证实验结果的可复现性 (Reproducibility)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
output_parser = StrOutputParser()

# 2. 定义提示词模板 (Prompt Engineering)
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant on {topic}.\n"
    "Answer the following question: {question}"
)

# ---------------------------------------------------------
# 方案 A：基础透传链 (Basic Passthrough Chain)
# 旧/基础写法对应图1：需要手动传入包含所有键的完整字典
# ---------------------------------------------------------
print("--- 执行方案 A (基础透传) ---")
basic_chain = RunnablePassthrough() | prompt | llm | output_parser

# 我们必须同时提供 topic 和 question
response_a = basic_chain.invoke({"topic": "movies", "question": "Tell me about The Godfather movie"})
print(f"Response A:\n{response_a}\n")


# ---------------------------------------------------------
# 方案 B：现代化注入链 (Modern Assignment Chain) - 核心实战！
# 对应图2及讲稿重点：利用 .assign() 隐式注入依赖
# ---------------------------------------------------------
print("--- 执行方案 B (使用 assign 动态注入) ---")

# 现代工程标准写法：
# 1. 第一个 RunnablePassthrough() 作为入口占位符接收用户的原始字典（只包含 question）。
# 2. .assign() 像拦截器一样，保留原始的 question，同时计算并注入 topic。
# 注：在实际科研或生产中，lambda x: "movies" 这里通常会替换成另一个大模型调用（例如分类器识别用户意图后动态决定 topic）。
smart_chain = (
    RunnablePassthrough() 
    | RunnablePassthrough.assign(topic=lambda x: "movies") 
    | prompt 
    | llm 
    | output_parser
)

# 此时，最终用户 (End-user) 只需要关心他们的问题，系统架构变得异常整洁 (Clean Architecture)
response_b = smart_chain.invoke({"question": "Tell me about Inception"})
print(f"Response B:\n{response_b}")

--- 执行方案 A (基础透传) ---
Response A:
"The Godfather" is a classic American crime film released in 1972, directed by Francis Ford Coppola. It is based on Mario Puzo's 1969 novel of the same name. The film is widely regarded as one of the greatest films in world cinema and has had a profound impact on the gangster genre.

The story follows the Corleone family, an Italian-American crime family in New York City, during the years after World War II. The patriarch, Vito Corleone, played by Marlon Brando, is the head of the family and runs a successful organized crime syndicate. The narrative explores themes of power, loyalty, family, and the American Dream.

Key characters include:
- Michael Corleone (played by Al Pacino), Vito's youngest son who initially wants nothing to do with the family business but becomes deeply involved as the story progresses.
- Sonny Corleone (James Caan), the hot-headed eldest son.
- Tom Hagen (Robert Duvall), the family lawyer and adopted son.
- Kay Adams (Diane Kea